In [3]:
from meshed.slabs import Slabs

from statistics import stdev
vol = stdev
should_turn_movement_sensor_on = lambda audio, light: vol(audio) * light > 50000

def make_a_slabs_iter():

    # Mocking the sensor readers
    audio_sensor_read = iter([[1, 2, 3], [-96, 87, -92], [320, -96, 99]]).__next__
    light_sensor_read = iter([126, 501, 523]).__next__
    movement_sensor_read = iter([None, None, True]).__next__

    return Slabs(
        # The first three components get data from the sensors.
        # The *_read objects are all callable, returning the next
        # chunk of data for that sensor, if any.
        audio=audio_sensor_read,
        light=light_sensor_read,
        movement=movement_sensor_read,
        # The next
        should_turn_movement_sensor_on = lambda audio, light: vol(audio) * light > 50000,
        human_presence_score = lambda audio, light, movement: movement and sum([vol(audio), light]),
        should_notify = lambda human_presence_score: human_presence_score and human_presence_score > 700,
        notify = lambda should_notify: print('someone is there') if should_notify else None
    )


si = make_a_slabs_iter()
next(si)  
    

{'audio': [1, 2, 3],
 'light': 126,
 'movement': None,
 'should_turn_movement_sensor_on': False,
 'human_presence_score': None,
 'should_notify': None,
 'notify': None}

In [4]:
next(si)

{'audio': [-96, 87, -92],
 'light': 501,
 'movement': None,
 'should_turn_movement_sensor_on': True,
 'human_presence_score': None,
 'should_notify': None,
 'notify': None}

In [5]:
next(si)

someone is there


{'audio': [320, -96, 99],
 'light': 523,
 'movement': True,
 'should_turn_movement_sensor_on': True,
 'human_presence_score': 731.1353726143957,
 'should_notify': True,
 'notify': None}

# Slabs for stamping press

In [57]:
from meshed.slabs import Slabs
from more_itertools import windowed

from typing import Callable

def filter_on_predicate(data, pred: Callable, event):
    if pred(event):
        return data
    else: 
        return None

def event_switch(event):
    return event and event['kind']=='event_a'

def make_a_slabs_iter():
    # Sensor readers
    accelerometer_sensor_read = iter([{'ch1':1, 'ch2':2, 'ch3':3}, {'ch1':-2, 'ch2':22, 'ch3':43},
                                      {'ch1':92, 'ch2':94, 'ch3':95}]).__next__
    plc_read = iter([None, {'kind':'event_a', 'bt':2}, {'kind':'event_b', 'tt':15}]).__next__

    return Slabs(
        accel=accelerometer_sensor_read,
        plc=plc_read,
        demux = lambda accel, plc: filter_on_predicate(accel, event_switch, plc),
        buffer = lambda demux: windowed(demux, n=2)
    )

s = make_a_slabs_iter()
next(s)
next(s)

{'accel': {'ch1': -2, 'ch2': 22, 'ch3': 43},
 'plc': {'kind': 'event_a', 'bt': 2},
 'demux': {'ch1': -2, 'ch2': 22, 'ch3': 43},
 'buffer': <generator object windowed at 0x7fa6058e94d0>}

In [52]:
s = make_a_slabs_iter()

In [54]:
list(next(s)['buffer'])

[('ch1', 'ch2'), ('ch2', 'ch3')]

In [36]:
from more_itertools import windowed

In [37]:
all_windows = windowed([1, 2, 3, 4, 5], 3)
list(all_windows)
[(1, 2, 3), (2, 3, 4), (3, 4, 5)]

[(1, 2, 3), (2, 3, 4), (3, 4, 5)]